In [6]:
import pandas as pd

# Charger les deux fichiers CSV dans des DataFrames
df2024 = pd.read_csv('player_stats_moyennes_roles_2024.csv')
df2025 = pd.read_csv('player_stats_moyennes_roles_2025.csv')

# Extraire la liste des joueurs de chaque saison (on suppose la colonne "player" = id unique)
joueurs_2024 = set(df2024['player'])
joueurs_2025 = set(df2025['player'])

# Trouver les joueurs présents dans les deux datasets
joueurs_en_commun = joueurs_2024 & joueurs_2025

# Afficher les joueurs communs (triés pour plus de lisibilité)
print("Joueurs présents dans les deux années :")
for joueur in sorted(joueurs_en_commun):
    print(joueur)
    
# Afficher le nombre de joueurs présents dans les deux années
print(f"Il y en a {len(joueurs_en_commun)} au total.")

Joueurs présents dans les deux années :
Alfajer
BeYN
Boaster
Boo
BuZz
CHICHOO
Chronicle
Flashback
Flex1n
Jinggg
JonahP
Knight
MaKo
Meteor
MiniBoo
RieNs
Smoggy
Wo0t
Zellsis
ZmjjKK
aspas
benjyfishy
d4v41
f0rsakeN
johnqt
leaf
nephh
nobody
something
trent
trexx
valyn
whzy
zekken
Il y en a 34 au total.


In [7]:

# On va comparer les stats principales pour les joueurs présents dans les deux années
# Statistiques à comparer (par exemple) :
stat_cols = ['rating', 'acs', 'kd_ratio', 'kast', 'adr', 'kpr', 'apr', 'fkpr', 'fdpr', 'hs_percent', 'cl_percent']

# On transforme les stats en float
for col in stat_cols:
    df2024[col] = pd.to_numeric(df2024[col], errors='coerce')
    df2025[col] = pd.to_numeric(df2025[col], errors='coerce')

# On crée deux DataFrames contenant uniquement les joueurs présents les deux années
df_24_common = df2024[df2024['player'].isin(joueurs_en_commun)].set_index('player')
df_25_common = df2025[df2025['player'].isin(joueurs_en_commun)].set_index('player')

# Pour chaque joueur, on calcule la différence de stats (2025 - 2024)
diff_stats = df_25_common[stat_cols] - df_24_common[stat_cols]

# On indique si chaque différence est positive (amélioration), négative (baisse), ou stable (0)
def compare_diff(val):
    if pd.isnull(val):
        return 'NA'
    elif val > 0:
        return '⬆️'
    elif val < 0:
        return '⬇️'
    else:
        return '→'

comparison = diff_stats.applymap(compare_diff)

# On assemble le tout dans un DataFrame pour affichage clair
result = pd.DataFrame({
    "Nom": df_24_common['player_name'],
})

for col in stat_cols:
    result[col + ' (Δ)'] = diff_stats[col]
    result[col + ' (Trend)'] = comparison[col]

# Afficher les évolutions par joueur
for joueur, row in result.iterrows():
    print(f"\n--- {row['Nom']} ({joueur}) ---")
    for col in stat_cols:
        diff = row[col + ' (Δ)']
        trend = row[col + ' (Trend)']
        print(f"{col}: {diff:+.2f} {trend}")




--- trexx (trexx) ---
rating: -0.36 ⬇️
acs: -67.90 ⬇️
kd_ratio: -0.51 ⬇️
kast: -9.00 ⬇️
adr: -45.30 ⬇️
kpr: -0.30 ⬇️
apr: -0.08 ⬇️
fkpr: -0.05 ⬇️
fdpr: +0.00 →
hs_percent: -3.00 ⬇️
cl_percent: -10.00 ⬇️

--- Chronicle (Chronicle) ---
rating: -0.12 ⬇️
acs: -18.40 ⬇️
kd_ratio: -0.16 ⬇️
kast: -3.00 ⬇️
adr: -14.90 ⬇️
kpr: -0.05 ⬇️
apr: -0.01 ⬇️
fkpr: -0.04 ⬇️
fdpr: +0.01 ⬆️
hs_percent: +3.00 ⬆️
cl_percent: -4.00 ⬇️

--- CHICHOO (CHICHOO) ---
rating: -0.29 ⬇️
acs: -63.30 ⬇️
kd_ratio: -0.34 ⬇️
kast: -7.00 ⬇️
adr: -31.00 ⬇️
kpr: -0.20 ⬇️
apr: -0.04 ⬇️
fkpr: -0.04 ⬇️
fdpr: +0.02 ⬆️
hs_percent: -8.00 ⬇️
cl_percent: -1.00 ⬇️

--- Meteor (Meteor) ---
rating: -0.13 ⬇️
acs: -22.10 ⬇️
kd_ratio: -0.36 ⬇️
kast: -3.00 ⬇️
adr: -16.50 ⬇️
kpr: -0.16 ⬇️
apr: +0.01 ⬆️
fkpr: +0.06 ⬆️
fdpr: -0.03 ⬇️
hs_percent: -1.00 ⬇️
cl_percent: -5.00 ⬇️

--- RieNs (RieNs) ---
rating: +0.02 ⬆️
acs: +11.90 ⬆️
kd_ratio: -0.03 ⬇️
kast: -2.00 ⬇️
adr: +3.90 ⬆️
kpr: +0.03 ⬆️
apr: +0.04 ⬆️
fkpr: -0.02 ⬇️
fdpr: -0.04 ⬇️
hs_percen

/tmp/ipykernel_3775/850196557.py:28: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  comparison = diff_stats.applymap(compare_diff)


In [9]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# Stats (hors ranking)
stat_cols = ['acs', 'kd_ratio', 'kast', 'adr', 'kpr', 'apr', 'fkpr', 'fdpr', 'hs_percent', 'cl_percent']

# Conversion en float
all_cols = ['rating'] + stat_cols
for col in all_cols:
    df2024[col] = pd.to_numeric(df2024[col], errors='coerce')
    df2025[col] = pd.to_numeric(df2025[col], errors='coerce')

# Joueurs communs
df_24 = df2024[df2024['player'].isin(joueurs_en_commun)].set_index('player')
df_25 = df2025[df2025['player'].isin(joueurs_en_commun)].set_index('player')

# Différences
diff = df_25[all_cols] - df_24[all_cols]

def categorize(x):
    if pd.isnull(x):
        return 'NA'
    if x > 0:
        return 'up'
    if x < 0:
        return 'down'
    return 'stable'

cat = diff.applymap(categorize)

# Variation centrale : le ranking
rank_var = cat['rating']

print("\n=== CHI-2 : dépendance entre variation du ranking et chaque stat ===\n")

for stat in stat_cols:
    contingency = pd.crosstab(rank_var, cat[stat], dropna=False)

    print(f"\nSTAT : {stat}")
    print(contingency)

    chi2, p, dof, expected = chi2_contingency(contingency)

    print(f"Chi²={chi2:.4f}  |  p={p:.6f}  |  dof={dof}")
    if p < 0.05:
        print("→ Dépendance significative avec le ranking.")
    else:
        print("→ Pas de dépendance claire.")



=== CHI-2 : dépendance entre variation du ranking et chaque stat ===


STAT : acs
acs     down  up
rating          
down      14   2
stable     1   1
up         4  12
Chi²=12.7053  |  p=0.001742  |  dof=2
→ Dépendance significative avec le ranking.

STAT : kd_ratio
kd_ratio  down  up
rating            
down        13   3
stable       1   1
up           2  14
Chi²=15.1849  |  p=0.000504  |  dof=2
→ Dépendance significative avec le ranking.

STAT : kast
kast    down  stable  up
rating                  
down      12       0   4
stable     2       0   0
up         4       1  11
Chi²=10.2000  |  p=0.037190  |  dof=4
→ Dépendance significative avec le ranking.

STAT : adr
adr     down  up
rating          
down      14   2
stable     1   1
up         4  12
Chi²=12.7053  |  p=0.001742  |  dof=2
→ Dépendance significative avec le ranking.

STAT : kpr
kpr     down  up
rating          
down      15   1
stable     1   1
up         4  12
Chi²=15.6795  |  p=0.000394  |  dof=2
→ Dépendance significa

/tmp/ipykernel_3775/2721305813.py:30: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cat = diff.applymap(categorize)
